In [ ]:
%matplotlib inline


In [ ]:
from pyLDLE2 import datasets
from pyLDLE2 import ldle_
import numpy as np
from umap import UMAP
from pyLDLE2 import visualize_
from matplotlib import pyplot as plt
import pandas as pd
import csv
from sklearn.decomposition import PCA
import pickle
from pyLDLE2 import visualize_all
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import TSNE
import flameplot as flameplot
from scipy.spatial import distance


## READ IN CSV

In [ ]:
### read in gene expression data (first 50 PC's)
with open('use.csv', 'rt') as f:
    reader = csv.reader(f)
    data_as_list = list(reader)
    
# read in cluster data (specific genes -- to visualize spatial locations)
with open('for_analysis.csv', 'rt') as f:
    reader = csv.reader(f, quoting = csv.QUOTE_NONNUMERIC)
    master = list(reader)

with open('vartab_ex1.csv', 'rt') as f:
    reader = csv.reader(f, quoting = csv.QUOTE_NONNUMERIC)
    grid_cell_data = list(reader)
    

grid_cell_data
    
    
### convert to np array
rna_seq_data = np.asarray(data_as_list)
master = np.array(master)
rna_seq_data = rna_seq_data.astype(float)

grid_cell_data = np.asarray(grid_cell_data)
grid_cell_data = grid_cell_data.astype(float)
# grid_cell_data

## EDIT DIRECTORIES FOR SAVING

In [ ]:
save_dir_root = 'C:\\Users\mirob\\Documents\\MORPH\\data\\LDLE\\rna\\' # where to save visualizations n stuff
dirpath = 'C:\\Users\mirob\\Documents\\MORPH\\' #where obsm.csv is stored or use.csv (I use use.csv)

## RUN LDLE ON RNA SEQUENCING DATA 

In [ ]:
rna_seq_data = rna_seq_data[:,:50] # first 991 PC's explain about 90% of variance, we used 50
rna_seq_data.shape
labelsMat = rna_seq_data[:,:2] #first two PC's
df = pd.DataFrame(rna_seq_data)
print(labelsMat)
df

In [ ]:
# optimal hyperparameters 
delta =  0.25 #0.25 .9
tau = 5 #5 50
k = 15 
k_tune = 7 

In [ ]:
# 'refine_algo_name': I am using gpa for rna seq data, not much difference for rgd
ldle_rna = ldle_.LDLE(local_opts={'algo':'LDLE', 'k':k, 'N': 25, 'k_nn':15 , 'delta':delta, 'tau':tau, 'k_tune':k_tune},
                  intermed_opts={'eta_min': 3, 'algo': 'mnm'},
                  global_opts={'alpha':0.01, "max_iter": 10, 'to_tear': False, 'color_tear': False, 'refine_algo_name': 'gpa'},
                  vis_opts={'cmap_interior': 'jet','cmap_boundary': 'jet','c': labelsMat[:,0], 'save_dir': save_dir_root},
                  verbose=True, debug=True)

In [ ]:
ldle_rna.fit(X=rna_seq_data)

## ADDITIONAL VISUALIZATIONS

In [ ]:
# # PATH TO SAVE VISUALIZATIONS
# save_path = save_dir_root+'/ldle_rna.dat'
# with open(save_path, "wb") as f:
#     pickle.dump([X, labelsMat*0, ldle2], f)
# print('Saved', save_path)

# DEFINE PATH TO SAVE ADDITIONAL EMBEDDINGS 
vis = visualize_.Visualize('C:\\Users\mirob\\Documents\\MORPH\\data\\LDLE\\additional_embeddings_rna\\')

In [ ]:
# VISUALIZE ALL 
# visualize_all.visualize('C:\\Users\mirob\\Documents\\MORPH\\data\\LDLE\\cluster\\ldle_rna.dat')

In [ ]:
# PCA
plt.scatter(labelsMat[:,0],labelsMat[:,1], c=labelsMat[:,0], cmap='jet')
plt.title("PCA")
plt.xlabel("PC 1")
plt.ylabel("PC 2")

In [ ]:
# UMAP: N_NEIGHBORS = 50; MIN_DIST = 0.25 (AFTER HYPER-PARAMETER TUNING)
n_neighbors = 60
min_dist = 0.25
print(n_neighbors, min_dist)
umap_obj = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2,
                random_state=42, n_epochs=500, metric='euclidean')
y_umap = umap_obj.fit_transform(rna_seq_data)
#         vis.global_embedding(y_umap, master[:,3], 'Blues', title='UMAP0_'+str(n_neighbors)+'_'+str(min_dist))
#         plt.show()
vis.global_embedding(y_umap, labelsMat[:,0], 'jet', title='UMAP1_'+str(n_neighbors)+'_'+str(min_dist))
plt.show()

In [ ]:
# TSNE: PERPLEXITY = 60, EE = 4 (AFTER HYPERPARAMETER TUNING )

perplexity = 60
ee = 4
print(perplexity, ee)
tsne_obj = TSNE(perplexity=perplexity, early_exaggeration=ee, n_components=2,
                metric='euclidean', random_state=42, n_iter=1000,
                n_jobs=-1, init='random')
y_tsne = tsne_obj.fit_transform(rna_seq_data)
vis.global_embedding(y_tsne, labelsMat[:,0], 'jet', title='t-SNE0_'+str(perplexity)+'_'+str(ee))
plt.show()

In [ ]:
# llE: N_NEIGHBORS = 100 (AFTER HYPERPARAMETER TUNING)
n_neighbors = 100
print(n_neighbors)
ltsa_obj = LocallyLinearEmbedding(method='standard', n_neighbors=n_neighbors, eigen_solver = 'dense',
                                  n_components=2)#, n_jobs=-1, random_state=42)
y_ltsa = ltsa_obj.fit_transform(rna_seq_data)
vis.global_embedding(y_ltsa, labelsMat[:,0], 'jet', title='LLE0_'+str(n_neighbors))
#     plt.savefig('LLE_1_TRANS.png', format='png', dpi=600, transparent=True)
plt.show()

## QUANTITATIVELY MEASURE DISTORTION AS IN LDLE PAPER (this can be optimized but for purpose of project it works)

In [ ]:
# ## UNCOMMENT TO: check distortion measurement (and code) works
# ## (if new embedding is a scalar of first, distortion should be 1)
# F = np.random.rand(10,9)
# F2 = 2.4*F
# DF = distance.squareform(distance.pdist(F))
# DF2 = distance.squareform(distance.pdist(F2))
# C  = np.sort(DF, axis=1)
# C2  = np.sort(DF2, axis=1)
# KNN = C[:,1:k+1]
# KNN2 = C2[:,1:k+1]
# Dk_num = np.divide(KNN,KNN2)
# Dk_den = np.divide(KNN2,KNN)
# Dk_num
# y_upper = np.amax(Dk_num,1)
# y_upper2 = np.amax(Dk_den,1)
# sam = y_upper*y_upper2
# sam

In [ ]:
D = distance.squareform(distance.pdist(grid_cell_data)) #if square data, change to square_data
# print(np.round(D, 1))
Du = distance.squareform(distance.pdist(y_umap))
# print(np.round(Du, 1))
Dt = distance.squareform(distance.pdist(y_tsne))
# print(np.round(Dt, 1))
Dl = distance.squareform(distance.pdist(y_ltsa))
# print(np.round(Dl, 1))
Dp = distance.squareform(distance.pdist(labelsMat)) #make sure you have PCA of rna or square run first
# print(np.round(Dp, 1))
Dld = distance.squareform(distance.pdist(ldle_grid.GlobalViews.y_init)) #if square data, change to ldle_square
# print(np.round(Dld,1))

In [ ]:
closest = np.sort(D, axis=1)
# print(closest)
closestu = np.sort(Du,axis=1)
# print(closestu)
closestt = np.sort(Dt,axis=1)
# print(closestt)
closestl = np.sort(Dl,axis=1)
# print(closestl)
closestp = np.sort(Dp,axis=1)
# print(closestp)
closestld = np.sort(Dld, axis=1)

In [ ]:
k = 100  # For each point, find the 100 closest points (or whatever #, doesn't matter)
knn_u= closestu[:,1:k+1]
knn = closest[:,1:k+1]
knn_t = closestt[:,1:k+1]
knn_l = closestl[:,1:k+1]
knn_p = closestp[:,1:k+1]
knn_ld = closestld[:,1:k+1]

In [ ]:
#LDLE VS OG
Dk_num = np.divide(knn_ld,knn)
Dk_den = np.divide(knn,knn_ld)
y_upper = np.amax(Dk_num,0)
y_upper2 = np.amax(Dk_den,0)
ldle_dk = y_upper*y_upper2
ldle_dk=ldle_dk
ldle_dk

In [ ]:
#UMAP VS OG
Dk_num = np.divide(knn_u,knn)
Dk_den = np.divide(knn,knn_u)
y_upper = np.amax(Dk_num,0)
y_upper2 = np.amax(Dk_den,0)
umap_dk = y_upper*y_upper2
umap_dk=umap_dk
umap_dk.shape


In [ ]:
#TSNE VS OG
Dk_num = np.divide(knn_t,knn)
Dk_den = np.divide(knn,knn_t)
y_upper = np.amax(Dk_num,0)
y_upper2 = np.amax(Dk_den,0)
tsne_dk = y_upper*y_upper2
tsne_dk= tsne_dk
tsne_dk

In [ ]:
#LLE VS OG
Dk_num = np.divide(knn_l,knn)
Dk_den = np.divide(knn,knn_l)
y_upper = np.amax(Dk_num,0)
y_upper2 = np.amax(Dk_den,0)
lle_dk = y_upper*y_upper2
lle_dk = lle_dk

In [ ]:
#PCA VS OG
Dk_num = np.divide(knn_p,knn)
Dk_den = np.divide(knn,knn_p)
y_upper = np.amax(Dk_num,0)
y_upper2 = np.amax(Dk_den,0)
pca_dk = y_upper*y_upper2
pca_dk = pca_dk

In [ ]:
import seaborn as sns
# g = sns.violinplot(umap_dk,cut=0)
# g.set(xlim=(0, None))
fig, ax = plt.subplots()
sns.set(rc={'figure.figsize':(8.7,7.27)})
sns.set(font_scale = 1.5)
sns.set_style('white', rc={
    'xtick.bottom': True,
    'ytick.left': False,
})
sns.set_style('ticks') 
labels = ["PCA", "UMAP",'LLE', 'tSNE', 'LDLE']
for i,arr in enumerate([pca_dk, umap_dk, lle_dk, tsne_dk,ldle_dk]):
    ax.violinplot(dataset=arr,positions=[i], showmedians=True, widths=0.8)
    plt.yscale("log") # for visualization purposes
#     plt.ylim(4000)
#     plt.ylabel()
    plt.title("Distortion (100 KNN)")
    print(np.median(arr))
    
    
ax.set(xticklabels=["filler", "PCA", "UMAP",'LLE', 'tSNE', 'LDLE'])


for i, arr in enumerate([pca_dk, umap_dk, lle_dk, tsne_dk,ldle_dk]):
    if i == 0:
        continue
    if i == 2:
        continue
# #     if i == 3:
#         continue
#     else:
    median = np.median(arr)
    maximum = np.max(arr)
    minimum = np.min(arr)
    plt.text((i-0.15), (median + .1), str(round(median, 2)), fontsize = 12)
    plt.text((i-0.15), (maximum), str(round(maximum, 2)), fontsize = 12)    
    plt.text((i-0.15), (minimum+np.log(.7)), str(round(minimum, 2)), fontsize = 12)
        
# for i, arr in enumerate([pca_dk, umap_dk, lle_dk, tsne_dk,ldle_dk]):
#     if i == 0:
#         continue
#     if i == 2:
#         continue
#     if i == 4:
#         continue
#     else:
#         median = np.median(arr)
#         maximum = np.max(arr)
#         minimum = np.min(arr)
#         plt.text((i"-0.15), (median+1), str(round(median, 2)), fontsize = 12)
#         plt.text((i-0.2), (maximum+20), str(round(maximum, 2)), fontsize = 12)    
#         plt.text((i-0.1), (minimum+np.log(.3)), str(round(minimum, 2)), fontsize = 12)
        
# for i, arr in enumerate([pca_dk, umap_dk, lle_dk, tsne_dk,ldle_dk]):
#     if i == 0:
#         continue
#     if i == 1:
#         continue
#     if i == 2:
#         continue
#     if i == 3:
#         continue
#     else:
#         median = np.median(arr)
#         maximum = np.max(arr)
#         minimum = np.min(arr)
#         plt.text((i-0.15), (median+1), str(round(median, 2)), fontsize = 12)
#         plt.text((i-0.2), (maximum+20), str(round(maximum, 2)), fontsize = 12)    
#         plt.text((i-.15), (minimum+np.log(.05)), str(round(minimum, 2)), fontsize = 12)

## QUALITATIVE MEASUREMENT OF DISTORTION:
# DONT RUN UNLESS YOU HAVE TO -- TAKES FOREVER

In [ ]:
# # Quantify UMAP vs. tSNE
scores1 = flameplot.compare(y_umap, y_tsne, n_steps=20)
# # Quantify UMAP vs. LDLE
scores2 = flameplot.compare(y_umap, ldle_grid.GlobalViews.y_final, n_steps=20)
# # Quantify LDLE vs. tSNE
scores3 = flameplot.compare(ldle_grid.GlobalViews.y_final, y_tsne, n_steps=20)

# # Plot
fig, ax = flameplot.plot(scores1, xlabel='UMAP (2D)', ylabel='tSNE (2D)')
fig, ax = flameplot.plot(scores2, xlabel='UMAP (2D)', ylabel='LDLE (2D)')
fig, ax = flameplot.plot(scores3, xlabel='LDLE (2D)', ylabel='tSNE (2D)')

In [ ]:
# Quantify LDLE vs. LLE
scores1 = flameplot.compare(ldle_grid.GlobalViews.y_final, y_ltsa, n_steps=20)
# Quantify LDLE vs. PCA
scores2 = flameplot.compare(ldle_grid.GlobalViews.y_final, labelsMat, n_steps=20)
# # Quantify LLE vs. PCA
scores3 = flameplot.compare(y_ltsa, labelsMat, n_steps=20)

# Plot
fig, ax = flameplot.plot(scores1, xlabel='LDLE (2D)', ylabel='LLE (2D)')
fig, ax = flameplot.plot(scores2, xlabel='LDLE (2D)', ylabel='PCA (2D)')
fig, ax = flameplot.plot(scores3, xlabel='LLE (2D)', ylabel='PCA (2D)')

In [ ]:
# import flameplot as flameplot

# Quantify PCA vs. tSNE
scores1 = flameplot.compare(labelsMat, y_tsne, n_steps=50)
# Quantify PCA vs. UMAP
scores2 = flameplot.compare(labelsMat, y_umap, n_steps=50)
# # Quantify LLE vs. tSNE
scores3 = flameplot.compare(y_ltsa, y_tsne, n_steps=50)
# # Quantify LLE vs. UMAP
scores4 = flameplot.compare(y_ltsa, y_umap, n_steps=50)

# Plot
fig, ax = flameplot.plot(scores1, xlabel='PCA (2D)', ylabel='tSNE (2D)')
fig, ax = flameplot.plot(scores2, xlabel='PCA (2D)', ylabel='UMAP (2D)')
fig, ax = flameplot.plot(scores3, xlabel='LLE (2D)', ylabel='tSNE (2D)')
fig, ax = flameplot.plot(scores4, xlabel='LLE (2D)', ylabel='UMAP (2D)')

In [ ]:
# # import flameplot as flameplot

# Quantify LDLE vs. X
scores1 = flameplot.compare(ldle_grid.GlobalViews.y_init, grid_cell_data, n_steps=20)
# Quantify UMAP vs. X
scores2 = flameplot.compare(y_umap, grid_cell_data, n_steps=20)
# # Quantify tSNE vs. X
scores3 = flameplot.compare(y_tsne, grid_cell_data, n_steps=20)
# # Quantify LLE vs. X
scores4 = flameplot.compare(y_ltsa, grid_cell_data, n_steps=20)
# # Quantify PCA vs. X
scores5 = flameplot.compare(labelsMat, grid_cell_data, n_steps=20)

# Plot
fig, ax = flameplot.plot(scores1, xlabel='LDLE (2D)', ylabel='X (50D)')
fig, ax = flameplot.plot(scores2, xlabel='UMAP (2D)', ylabel='X (50D)')
fig, ax = flameplot.plot(scores3, xlabel='tSNE (2D)', ylabel='X (50D)')
fig, ax = flameplot.plot(scores4, xlabel='LLE (2D)', ylabel='X (50D)')
fig, ax = flameplot.plot(scores5, xlabel='PCA (2D)', ylabel='X (50D)')

# SQUARE WITH TWO HOLES
## run LDLE

In [ ]:
square_data, labelsMat, ddX = datasets.Datasets().squarewithtwoholes()
# labelsMat
square_data

In [ ]:
# print(square_data)
# vartab_ex1

In [ ]:
# The supplied options would override the default options
ldle_square = ldle_.LDLE(local_opts={'algo':'LDLE', 'k': 35},
                  intermed_opts={'algo':'mnm', 'eta_min': 3},
                  global_opts={'max_iter': 8},
                  vis_opts={'c': labelsMat[:,0],'save_dir': save_dir_root},
                  verbose=True, debug=True)

In [ ]:
ldle_square.fit(X=square_data)

## ADDITIONAL VISUALIZATIONS

In [ ]:
vis = visualize_.Visualize('C:\\Users\mirob\\Documents\\MORPH\\data\\LDLE\\additional_embeddings_square\\')

In [ ]:
#PCA
# PCA
pca = PCA(n_components=2)
labelsMat = pca.fit_transform(square_data)
plt.scatter(labelsMat[:,0],labelsMat[:,1], c=labelsMat[:,0], cmap='summer')
plt.title("PCA")
plt.xlabel("PC 1")
plt.ylabel("PC 2")


In [ ]:
pca.explained_variance_ratio_

In [ ]:
#UMAP
n_neighbors = 50
min_dist = 0.25
print(n_neighbors, min_dist)
umap_obj = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2,
                random_state=42, n_epochs=500, metric='euclidean')
y_umap = umap_obj.fit_transform(square_data)
#         vis.global_embedding(y_umap, master[:,3], 'Blues', title='UMAP0_'+str(n_neighbors)+'_'+str(min_dist))
#         plt.show()
vis.global_embedding(y_umap, labelsMat[:,0], 'summer', title='UMAP1_'+str(n_neighbors)+'_'+str(min_dist))
plt.show()

In [ ]:
#tSNE
perplexity = 60
ee = 4
print(perplexity, ee)
tsne_obj = TSNE(perplexity=perplexity, early_exaggeration=ee, n_components=2,
                metric='euclidean', random_state=42, n_iter=1000,
                n_jobs=-1, init='random')
y_tsne = tsne_obj.fit_transform(square_data)
vis.global_embedding(y_tsne, labelsMat[:,0], 'summer', title='t-SNE0_'+str(perplexity)+'_'+str(ee))
plt.show()
#         vis.global_embedding(y_tsne, labelsMat[:,1], 'jet', title='t-SNE1_'+str(perplexity)+'_'+str(ee))
#         plt.show()

In [ ]:
# llE: N_NEIGHBORS = 100 (AFTER HYPERPARAMETER TUNING)
n_neighbors = 100
print(n_neighbors)
ltsa_obj = LocallyLinearEmbedding(method='ltsa', n_neighbors=n_neighbors, eigen_solver = 'dense',
                                  n_components=2, n_jobs=-1, random_state=42)
y_ltsa = ltsa_obj.fit_transform(square_data)
vis.global_embedding(y_ltsa, labelsMat[:,0], 'summer', title='LLE0_'+str(n_neighbors))
#     plt.savefig('LLE_1_TRANS.png', format='png', dpi=600, transparent=True)
plt.show()

In [ ]:
# Repeat distortion here (from above) if want to quantify distortion on square with two holes

## GRID CELL

In [ ]:
save_dir_root = 'C:\\Users\mirob\\Documents\\MORPH\\data\\LDLE\\grid\\' # where to save visualizations n stuff
grid_cell_data.shape

In [ ]:
plt.scatter(grid_cell_data[:,0],grid_cell_data[:,1],c=grid_cell_data[:,0], cmap='jet')

In [ ]:
pca = PCA(n_components=2)
labelsMat = pca.fit_transform(grid_cell_data)
plt.scatter(labelsMat[:,0],labelsMat[:,1],c=labelsMat[:,0], cmap='jet')
plt.title("PCA")
plt.xlabel("PC 1 (52.25%)")
plt.ylabel("PC 2 (47.75%)" )

In [ ]:
pca.explained_variance_ratio_*100

In [ ]:
# X = grid_cell_data - np.mean(grid_cell_data,axis=0)[np.newaxis,:]
# X = X / (np.std(X,axis=0)[np.newaxis,:] + 1e-12)
# X_new = X
# X_new = X_new / np.max(np.abs(X_new))
# plt.scatter(X_new[:,0],X_new[:,1],c=X_new[:,0], cmap='summer')

In [ ]:
min_pose = np.min(labelsMat[:,1])
min_light = np.min(labelsMat[:,0])
max_pose = np.max(labelsMat[:,1])
max_light = np.max(labelsMat[:,0])

In [ ]:
N = grid_cell_data.shape[0]
ddX = np.zeros(N)
for k in range(N):
    ddX1 = np.min([labelsMat[k,0]-min_light, max_light-labelsMat[k,0]])
    ddX2 = np.min([labelsMat[k,1]-min_pose, max_pose-labelsMat[k,1]])
    ddX[k] = np.min([ddX1, ddX2])

In [ ]:
# ddX

In [ ]:
# pca = PCA(n_components=2)
# labelsMat = pca.fit_transform(yonatan_nogrid)
# # fig = plt.figure(figsize=(12, 12))
# # ax = fig.add_subplot(projection='3d')
# plt.scatter(labelsMat[:,0],labelsMat[:,1], c=labelsMat[:,0], cmap='summer')
# plt.title("PCA")
# plt.xlabel("PC 1")
# plt.ylabel("PC 2")

In [ ]:
vis = visualize_.Visualize('C:\\Users\mirob\\Documents\\MORPH\\data\\LDLE\\additional_embeddings_grid\\')
n_neighbors = 100
# print(grid_cell_data)
ltsa_obj = LocallyLinearEmbedding(method='ltsa', n_neighbors=n_neighbors, eigen_solver = 'dense',
                                  n_components=2, n_jobs=-1, random_state=42)
y_ltsa = ltsa_obj.fit_transform(grid_cell_data)
vis.global_embedding(y_ltsa, labelsMat[:,0], 'jet', title='LLE0_'+str(n_neighbors))
#     plt.savefig('LLE_1_TRANS.png', format='png', dpi=600, transparent=True)
plt.show()

In [ ]:
perplexity = 60
ee = 4
print(perplexity, ee)
tsne_obj = TSNE(perplexity=perplexity, early_exaggeration=ee, n_components=2,
                metric='euclidean', random_state=42, n_iter=1000,
                n_jobs=-1, init='random')
y_tsne = tsne_obj.fit_transform(grid_cell_data)
vis.global_embedding(y_tsne, labelsMat[:,0], 'jet', title='t-SNE0_'+str(perplexity)+'_'+str(ee))
plt.show()

In [ ]:
n_neighbors = 100
min_dist = .9
print(n_neighbors, min_dist)
umap_obj = UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2,
                random_state=42, n_epochs=500, metric='euclidean')
y_umap = umap_obj.fit_transform(grid_cell_data)
#         vis.global_embedding(y_umap, master[:,3], 'Blues', title='UMAP0_'+str(n_neighbors)+'_'+str(min_dist))
#         plt.show()
vis.global_embedding(y_umap, labelsMat[:,0], 'jet', title='UMAP1_'+str(n_neighbors)+'_'+str(min_dist))
plt.show()

In [ ]:
# The supplied options would override the default options
ldle_grid = ldle_.LDLE(local_opts={'algo':'LDLE', 'k': 10, 'k_nn': 10, 'k_tune':7, "delta":.25, "tau":20, 'N':15},
                  intermed_opts={'algo':'best', 'eta_min': 5},
                  global_opts={'max_iter': 8, 'refine_algo_name': 'gpa'},
                  vis_opts={'cmap_interior': 'jet', 'cmap_boundary':'jet', 'c': labelsMat[:,0],'save_dir': save_dir_root},
                  verbose=True, debug=True)

In [ ]:
ldle_grid.fit(X=grid_cell_data)

In [ ]:
ldle_grid.GlobalViews.y_final


In [ ]:
ldle_grid.GlobalViews.y_init